In [37]:
import glob
import cv2
import numpy as np
import csv
from collections import Counter
import argparse
import os
import math

In [44]:
DEFAULT_VALUE = int(255)

NL = int(30)
start_superpixels = int(500)
last_superpixels = int(30)
csv_sizes = []
reduction_factor = math.pow(float(last_superpixels) / start_superpixels, 1. / (NL - 1))
for level in range(NL):
    csv_sizes = csv_sizes + [int(round(start_superpixels * math.pow(reduction_factor, level)))]

dataset = '/data/jantina/ML-Superpixels/Datasets/WAPA_RFM4'
    
path_names = dataset.split('/')
if path_names[-1] == '':
    path_names = path_names[:-1]
directorio = path_names[-1]

sparse_dir = os.path.join('/data/jantina/ML-Superpixels/WAPA_RFM4/sparse_GT')
out_dir = os.path.join('/data/jantina/ML-Superpixels/WAPA_RFM4/augmented_GT')
superpixels_dir = os.path.join('/data/jantina/ML-Superpixels/WAPA_RFM4/superpixels')

folders = ['test', 'train']

In [ ]:
# Execute superpixel generation
size_sup_string = " "
for size in csv_sizes:
    size_sup_string = size_sup_string + str(size) + " "

%cd /data/jantina/ML-Superpixels
# Generate superpixels
os.system("sh /home/jantina/CoralNet/Superpixels/generate_superpixels/generate_superpixels.sh " + dataset + size_sup_string)

/data/jantina/ML-Superpixels
318 images total ...
On average, 1.10685 seconds needed ...
318 images total ...
On average, 1.10614 seconds needed ...
318 images total ...
On average, 1.08354 seconds needed ...
318 images total ...
On average, 1.09619 seconds needed ...
318 images total ...
On average, 1.08043 seconds needed ...
318 images total ...
On average, 1.08765 seconds needed ...
318 images total ...
On average, 0.891684 seconds needed ...
318 images total ...
On average, 0.900087 seconds needed ...
318 images total ...
On average, 0.90788 seconds needed ...
318 images total ...
On average, 0.906477 seconds needed ...
318 images total ...
On average, 0.91267 seconds needed ...
318 images total ...
On average, 0.913355 seconds needed ...
318 images total ...
On average, 0.905794 seconds needed ...
318 images total ...
On average, 0.90547 seconds needed ...
318 images total ...
On average, 0.816534 seconds needed ...
318 images total ...
On average, 0.818457 seconds needed ...
318 

Killed


318 images total ...
On average, 1.15926 seconds needed ...
318 images total ...
On average, 1.14019 seconds needed ...
318 images total ...
On average, 1.13004 seconds needed ...
318 images total ...
On average, 1.12534 seconds needed ...
318 images total ...
On average, 1.68929 seconds needed ...
318 images total ...
On average, 1.91746 seconds needed ...
318 images total ...
On average, 1.73274 seconds needed ...
318 images total ...
On average, 1.38822 seconds needed ...
318 images total ...
On average, 1.09091 seconds needed ...
318 images total ...
On average, 1.03846 seconds needed ...
318 images total ...
On average, 1.1988 seconds needed ...
318 images total ...
On average, 0.981029 seconds needed ...
318 images total ...
On average, 1.04072 seconds needed ...
318 images total ...
On average, 0.744728 seconds needed ...
318 images total ...
On average, 0.759675 seconds needed ...
318 images total ...
On average, 0.756258 seconds needed ...
318 images total ...
On average, 0.77

In [39]:
class Superpixel:
    def __init__(self, maxlength=10000): #10000000
        self.lista_x = np.zeros([maxlength], dtype=np.int16)
        self.lista_y = np.zeros([maxlength], dtype=np.int16)
        self.lista_x[:]=-1
        self.lista_y[:]=-1
        self.index = 0

    def add(self, value_x, value_y):
        if self.index >= len(self.lista_x):
            #make it bigger
            new_size = len(self.lista_x)*10
            lista_x_new = np.zeros([new_size], dtype=np.int16)
            lista_y_new = np.zeros([new_size], dtype=np.int16)
            lista_x_new[:] = -1
            lista_y_new[:] = -1

            lista_x_new[:len(self.lista_x)] = self.lista_x
            lista_y_new[:len(self.lista_y)] = self.lista_y
            self.lista_x = lista_x_new
            self.lista_y = lista_y_new

        self.lista_x[self.index]=value_x
        self.lista_y[self.index]=value_y
        self.index += 1
        # print(len(lista_y_new))

    def clean(self):
        self.lista_x = self.lista_x[self.lista_x>=0]
        self.lista_y = self.lista_y[self.lista_y>=0]

In [40]:
# Given a superpixel and a GT image, returns the label value of the superpixel
def label_mayoria_x_y(superpixel, gt):
    # pixel label values of the superpixels
    pixel_values = gt[superpixel.lista_x.astype(int), superpixel.lista_y.astype(int)]
    # pixel label values of the superpixels excluding the default value
    values_labels = pixel_values[pixel_values < DEFAULT_VALUE]
    # Returns the value which appears the most
    if len(values_labels) == 0:
        return DEFAULT_VALUE
    else:
        count = Counter(values_labels)
        return count.most_common()[0][0]

In [41]:
# Given a csv file with segmentations (csv_name) and a sparse GT image (gt_name), returns Superpixel-augmented GT image
def image_superpixels_gt(csv_name, gt_name, n_superpixels):
    print(gt_name)
    gt = cv2.imread(gt_name, 0)
    blank_image = np.zeros((gt.shape[0], gt.shape[1], 1), np.uint8)

    superpixels = {}

    for i in range(n_superpixels*2):
        superpixels[str(i)] = Superpixel()


    # For each csv segmentation file, creates the Superpixel class
    with open(csv_name, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        i = 0

        for row in spamreader:
            fila = row[0].split(',')
            fila_count = len(fila)
            for j in range(fila_count):
                superpixel_index = fila[j]
                # The pixel here is (i, j). (superpixel_index) is the segmentation which the pixel belongs to
                # Add the pixel  to the Superpixel instance
                superpixels[superpixel_index].add(i, j)

            i = i + 1

    # For each superpixel, gets its label value and writes it into the image to return
    for index in range(len(superpixels)):
        superpixels[str(index)].clean()
        label_superpixel = label_mayoria_x_y(superpixels[str(index)], gt)
        blank_image[superpixels[str(index)].lista_x.astype(int), superpixels[str(index)].lista_y.astype(int)] = int(
            label_superpixel)

    return blank_image

In [42]:
%cd /data/jantina/ML-Superpixels

def generar_augmentedGT():
    for folder in folders:

        in_folder = os.path.join(sparse_dir, folder)
        out_folder = os.path.join(out_dir, folder)
        superpixels_folder = os.path.join(superpixels_dir, folder)

        if not os.path.exists(out_folder):
            os.makedirs(out_folder)
            
        print(in_folder + '/*.' + 'png')
  
        for filename in glob.glob(in_folder + '/*.' + 'png'):  # imagenes test a crear patches
            gt_name = filename.split('/')[-1]
            gt_filename = os.path.join(out_folder, gt_name)
            print('In here')
            # For each different segmentation generated
            for index in range(len(csv_sizes)):
                print(csv_sizes[index])

                csv_name = os.path.join(superpixels_folder, 'superpixels_' + str(csv_sizes[index]),
                                        gt_name.replace('.' + 'png', '') + '.csv')

                if index == 0:
                    # creates the first one (it has to be the more detailed one, the segmentation with more segments)
                    image_gt_new = image_superpixels_gt(csv_name, filename, csv_sizes[index])
                else:
                    # Mask it with the less detailed segmentations in order to fill the areas with no valid labels
                    image_gt_new_low = image_superpixels_gt(csv_name, filename, csv_sizes[index])
                    image_gt_new[image_gt_new == DEFAULT_VALUE] = image_gt_new_low[image_gt_new == DEFAULT_VALUE]
                    print(image_gt_new.shape)
                # cv2.imwrite(gt_filename.replace(gt_name, gt_name.replace('.png', '_'+str(csv_sizes[index])+'.png')),image_gt_new)

            # out_dir
            cv2.imwrite(gt_filename, image_gt_new)

/data/jantina/ML-Superpixels


In [43]:
generar_augmentedGT()
print('GENERATION COMPLETED')


/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/*.png
In here
500
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
454
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000, 1)
412
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000, 1)
374
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000, 1)
339
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000, 1)
308
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000, 1)
279
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000, 1)
254
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 14.5-B (P8072827).png
(3000, 4000,

(3000, 4000, 1)
209
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
190
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
172
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
156
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
142
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
129
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
117
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
106
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 15.5-B (P8072825).png
(3000, 4000, 1)
96
/data/jantina/ML-Superpixels/

(3000, 4000, 1)
79
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
72
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
65
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
59
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
54
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
49
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
44
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
40
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 16.0-A (P8072778).png
(3000, 4000, 1)
36
/data/jantina/ML-Superpixels/WAPA_RFM

(3000, 4000, 1)
30
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.0-A (P8072780).png
(3000, 4000, 1)
In here
500
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
454
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
(3000, 4000, 1)
412
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
(3000, 4000, 1)
374
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
(3000, 4000, 1)
339
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
(3000, 4000, 1)
308
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
(3000, 4000, 1)
279
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 17.5-B (P8072821).png
(3000, 4000, 1)
254
/data/jantina/ML-Superpixels/WAPA_RFM

(3000, 4000, 1)
209
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
190
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
172
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
156
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
142
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
129
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
117
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
106
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/test/2020-08-07-ADELUP-B-T4 18.5-B (P8072819).png
(3000, 4000, 1)
96
/data/jantina/ML-Superpixels/

...

(3000, 4000, 1)
412
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
374
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
339
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
308
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
279
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
254
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
230
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
209
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 12.5-B (PB070178).png
(3000, 4000, 1)
190
/data/jantina/ML-Sup

(3000, 4000, 1)
172
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
156
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
142
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
129
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
117
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
106
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
96
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
87
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 13.5-B (PB070176).png
(3000, 4000, 1)
79
/data/jantina/ML-Superp

(3000, 4000, 1)
72
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
65
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
59
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
54
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
49
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
44
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
40
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
36
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 14.5-B (PB070174).png
(3000, 4000, 1)
33
/data/jantina/ML-Superpixels/

(3000, 4000, 1)
30
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 15.5-B (PB070172).png
(3000, 4000, 1)
In here
500
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
454
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
(3000, 4000, 1)
412
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
(3000, 4000, 1)
374
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
(3000, 4000, 1)
339
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
(3000, 4000, 1)
308
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
(3000, 4000, 1)
279
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 1.5-B (PB070200).png
(3000, 4000, 1)
254
/data/jantina/ML-Superpixels/WAPA_RF

(3000, 4000, 1)
230
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
209
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
190
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
172
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
156
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
142
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
129
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
117
/data/jantina/ML-Superpixels/WAPA_RFM3/sparse_GT/train/2021-11-07-ADELUP-B-T4 16.5-B (PB070170).png
(3000, 4000, 1)
106
/data/jantina/ML-Sup